In [26]:
import numpy as np

# Euler Angles and Rotation Matrix
- Direct problem: euler -> rotm
- Inverse problem: rotm -> euler

![EULER-ANGLES-RPY](images/roll-pitch-yaw.jpg)

In [27]:
# Calculates Rotation Matrix given euler angles in the order of YAW(psi), PITCH(theta), ROLL(phi)
def eulerAnglesToRotationMatrix(theta):
    '''
    :param: theta: np array 
        array of angles (radians) in the order of YAW(psi), PITCH(theta), ROLL(phi)

    YAW = rotation around the x axis
    PITCH = rotation around the y axis
    ROLL = rotation around the z axis

    :return: 3 by 3 np array
        Return 3x3 rotation matrix
    '''
    #yaw rotation
    R_x = np.array([[1,         0,                  0                   ],
                    [0,         np.cos(theta[0]), -np.sin(theta[0]) ],
                    [0,         np.sin(theta[0]), np.cos(theta[0])  ]
                    ])
    #pitch rotation
    R_y = np.array([[np.cos(theta[1]),    0,      np.sin(theta[1])  ],
                    [0,                     1,      0                   ],
                    [-np.sin(theta[1]),   0,      np.cos(theta[1])  ]
                    ])
    #roll rotation
    R_z = np.array([[np.cos(theta[2]),    -np.sin(theta[2]),    0],
                    [np.sin(theta[2]),    np.cos(theta[2]),     0],
                    [0,                     0,                      1]
                    ])

    R = np.dot(R_z, np.dot( R_y, R_x ))

    return R


In [28]:
angles_1 = np.array([0, 45, 90]) * (np.pi/180)
print(angles_1)

[0.         0.78539816 1.57079633]


In [29]:
R = eulerAnglesToRotationMatrix(angles_1)
print(R)

[[ 4.32978028e-17 -1.00000000e+00  4.32978028e-17]
 [ 7.07106781e-01  6.12323400e-17  7.07106781e-01]
 [-7.07106781e-01  0.00000000e+00  7.07106781e-01]]


In [30]:
def isRotationMatrix(R, precision = 1e-4):
    """
    Checks if a matrix is a rotation matrix
    :param R: np.array matrix of 3 by 3
    :param precision: float
        precision to respect to accept a zero value in identity matrix check (default is 1e-4)
    :return: True or False
        Return True if a matrix is a rotation matrix, False if not
    """
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype=R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < precision

def rotationMatrixToEulerAngles(R, precision = 1e-4):
    """
    Computes the Tait–Bryan Euler () angles from a Rotation Matrix.
    Also checks if there is a gymbal lock and eventually use an alternative formula
    :param R: np.array
        3 x 3 Rotation matrix
    :param precision: float
        precision to respect to accept a zero value in identity matrix check (default is 1e-4)
    :return: (yaw, pitch, roll) tuple of float numbers
        Euler angles in radians in the order of YAW, PITCH, ROLL
    """
    # Calculates Tait–Bryan Euler angles from a Rotation Matrix
    assert (isRotationMatrix(R, precision))  # check if it's a Rmat

    sy = np.sqrt(R[0, 0] * R[0, 0] + R[1, 0] * R[1, 0]) # assert that sqrt(R11^2 + R21^2) != 0
    singular = sy < precision

    if not singular:  #if not in a singularity, use the standard formula
        x = np.arctan2(R[2, 1], R[2, 2])  # atan2(R31, R33) -> YAW, angle PSI
        
        # atan2(-R31, sqrt(R11^2 + R21^2)) -> PITCH, angle delta
        y = np.arctan2(-R[2, 0], sy)

        z = np.arctan2(R[1, 0], R[0, 0]) #atan2(R21,R11) -> ROLL, angle phi

    else:  # if in gymbal lock, use different formula for yaw, pitch roll
        x = np.arctan2(-R[1, 2], R[1, 1])
        y = np.arctan2(-R[2, 0], sy)
        z = 0

    return np.array([x, y, z]) # returns YAW, PITCH, ROLL


In [31]:
angles = rotationMatrixToEulerAngles(R) * 180/np.pi
print(f"YAW:{angles[0]}\nPITCH:{angles[1]}\nROLL:{angles[2]}")

YAW:0.0
PITCH:45.0
ROLL:90.0


In [32]:
#let's try a singularity point where pitch = 90°
sing_angl = np.array([45,90,45]) * np.pi/180
R_sing = eulerAnglesToRotationMatrix(sing_angl)
print(R)
rev_angl = rotationMatrixToEulerAngles(R_sing) * 180/np.pi
print(f"YAW:{rev_angl[0]}\nPITCH:{rev_angl[1]}\nROLL:{rev_angl[2]}")


[[ 4.32978028e-17 -1.00000000e+00  4.32978028e-17]
 [ 7.07106781e-01  6.12323400e-17  7.07106781e-01]
 [-7.07106781e-01  0.00000000e+00  7.07106781e-01]]
YAW:-0.0
PITCH:90.0
ROLL:0.0
